**Question**

Write a function to calculate the change in computation time and prediction accuracy per principal component added to the feature set for kNN. Have your function call be pcaCompare(X,Y,rows,k,pcs), where X, Y, and k are the usual inputs to kNN, rows is the rows of the data we want to test, and pcs is the maximum number of principal components tested. Have it return a 3 column matrix representing the change in time and accuracy (not as a percentage) that occurs by adding this PC as a feature (eg. negative indicates the value went down when it was added) in the first and second columns, respectively, and the variance of the PC in the third, where the PC number is the row number.**bold text**


In [0]:
def kNN(X,Y,newx,k,regress=True,allK=False,leave1out=False,scaleX=True,scaler='standard'):

  import warnings
  warnings.filterwarnings('ignore')

  import numpy as np
  import pandas as pd

  from sklearn.neighbors import KNeighborsClassifier 
  from sklearn.neighbors import KNeighborsRegressor

  from sklearn.preprocessing import StandardScaler
  #from sklearn.preprocessing import MinMaxScaler
  #from sklearn.preprocessing import RobustScaler

  from statistics import mean 
  from statistics import mode
  from collections import Counter

  def kNNtype(neighbs,regress):
    if regress:
      knn = KNeighborsRegressor(n_neighbors=neighbs)
    else:
      knn = KNeighborsClassifier(n_neighbors=neighbs)
    return knn

  if scaler != 'standard':
    scaler = scaler
  else:
    scaler = StandardScaler()

# Update: for row subsets/test sets in hw; fitting the scaing function should be done separate
# so the same can be applied to train and test data (or X and newx) 
  if scaleX == True:
    # scale should be fit to X/train
    scaler.fit(X)
    columns_X = [list(X.columns)]
    for feature in columns_X:
      X[feature] = scaler.transform(X[feature])
      newx[feature] = scaler.transform(newx[feature])

  knn_all = pd.DataFrame()
  if allK == True:
    if leave1out == True:
      nn_all = []
      for j in list(newx.index.values.tolist()):
      #for j in list(Y.index.values.tolist()):
        knn_row = []
        knn = kNNtype(k+1,regress)
        knn.fit(X, Y)
        test = pd.DataFrame(newx.loc[j,:])
        nn = knn.kneighbors(test.T)[1][0]
        for i in range(2,k+1):
          nn1 = nn[1:i] # leave one out
          test = list(Y.iloc[nn1])
          if regress:
            test = mean(test)
          else:
            c = Counter(test)
            l = list(c.values())
            ind = l.index(max(c.values()))
            test = list(c.keys())[ind]
            # count number of times the max class occurs and if there is a tie
            # choose the second class with the max if index is even
            if (l.count(max(l))) > 1 and (j % 2 !=0):
              l[ind] = 0
              ind = l.index(max(c.values()))
              test = list(c.keys())[ind]

          knn_row.append(test)
        knn_row = pd.DataFrame(knn_row)
        knn_all = [knn_all, knn_row]
        knn_all = pd.concat(knn_all,axis=1, ignore_index=True)
        nn_all.append(list(nn1))
      nn_all = np.array(nn_all)
    else:
        for i in range(1,k+1):
          knn = kNNtype(i,regress)
          knn.fit(X, Y)
          test = knn.predict(newx)
          knn_row = pd.DataFrame(test).T
          knn_all = [knn_all, knn_row]
          knn_all = pd.concat(knn_all,axis=0, ignore_index=True)
        nn_all = knn.kneighbors(newx)[1]
  else:
    if leave1out == True:
      knn_row = []
      for j in list(newx.index.values.tolist()):
      #for j in list(Y.index.values.tolist()):
        knn = kNNtype(k,regress)
        knn.fit(X, Y)
        test = pd.DataFrame(newx.loc[j,:])
        nn = knn.kneighbors(test.T)[1][0]
        nn1 = nn[1:len(nn)]

        test = list(Y.iloc[nn1])
        if regress:
            test = mean(test)
        else:
          c = Counter(test)
          l = list(c.values())
          ind = l.index(max(c.values()))
          test = list(c.keys())[ind]
          # count number of times the max class occurs and if there is a tie
          # choose the second class with the max if index is even
          if (l.count(max(l))) > 1 and (j % 2 !=0):
            l[ind] = 0
            ind = l.index(max(c.values()))
            test = list(c.keys())[ind]

        knn_row.append(test)
      knn_all = pd.DataFrame(knn_row).T
      nn_all = nn1
    else:
        knn = kNNtype(k,regress)
        knn.fit(X, Y)
        test = knn.predict(newx)
        knn_all = pd.DataFrame(test)
        nn_all = knn.kneighbors(newx)[1]

  return knn_all, nn_all

In [0]:
def findOverallLoss(ypreds,Y):
  err = abs(ypreds.reset_index(drop=True).T - Y.reset_index(drop=True).values.squeeze())
  MAPE = err.mean(axis=1)
  return MAPE

In [0]:
def pcaCompare(X,Y,rows,k, pcs):
  from sklearn.decomposition import PCA
  from sklearn.preprocessing import StandardScaler
  from sklearn.metrics import r2_score

  train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=rows, random_state=1)
  scaler = StandardScaler()
  scaler.fit(train_X)

  train_X = scaler.transform(train_X)
  test_X = scaler.transform(test_X)
  #newx = scaler.transform(rows)
  x = 1

  time_pca = []
  var = []
  acc = []
  basetime = 0 

  time_acc_var = pd.DataFrame(columns=['Time','Accuracy','Variance'])
  start = time.time()
  out,neighbors = kNN(train_X, train_Y, test_X, k,regress=True,scaleX = False) # regress = True 
  basetime = time.time() - start
  print("--- kNN without pca %s seconds ---" % basetime)
  knnscore = r2_score(test_Y, out)
  print("--- kNN score without pca %s ---" % knnscore)

  while x <= pcs:
    pca = PCA(n_components = x)
    pca.fit(train_X)
   # print("--- %s pc ---" % (pca.n_components_))
    X_pca = pca.transform(train_X)
    newx_pca = pca.transform(test_X)
    start_time = time.time()
    knnout, nn = kNN(X_pca, train_Y, newx_pca, k,regress=True,scaleX=False)
   # print("--- %s seconds ---" % (time.time() - start_time))
    pcatime = (time.time() - start_time)
    time_pca.append(pcatime - basetime)
    #print("--- %s accuracy score ---" % r2_score(test_Y,knnout))
    pcascore = r2_score(test_Y,knnout) # change to change in acc 
    acc.append(knnscore - pcascore)
   # print("--- %s variance ---" % pca.explained_variance_ratio_[x - 1])
    var.append(pca.explained_variance_ratio_[x - 1])
    x += 1

  for i in range(len(acc)):
    time_acc_var.loc[i+1, 'Time'] = time_pca[i]
    time_acc_var.loc[i+1, 'Accuracy'] = acc[i]
    time_acc_var.loc[i+1, 'Variance'] = var[i]
    
  return time_acc_var, knnscore, basetime

    # writing to data frame
    

Write a wrapper function for pcaCompare to find the best number of PCs for the optimal runtime and a loss no greater than maxLoss. Have your function call be pcaOptimize(X,Y,rows,k,pcs,maxLoss), where X, Y, rows, k, and pcs are the same as above, and maxLoss is the maximum acceptable average loss as a percentage of the loss without using pca (eg. maxLoss=1.1 then it can be 10% higher.)

In [0]:
def pcaOptimize(X,Y, rows,k,pcs,maxLoss):
    table, knnscore, knntime = pcaCompare(X,Y, rows, k, pcs)
    scorerange = (maxLoss - int(maxLoss)) * knnscore
    acceptable = knnscore - scorerange
    print(acceptable)
    answer  = 0 
    timetable = []
    rslt = table.loc[table['Accuracy'] >= acceptable]
    print(rslt)
    
    timetable = rslt['Time']
    answer = min(timetable)
    print(answer)
    x = 1
    for row in range(len(timetable)):
      if (answer == timetable[row + 1]):
        print("Best PC for optimization is : %s " %x)
      x += 1
  
    



In [0]:
for i in retur:
  print(i)
  print('space')

       Time   Accuracy   Variance
1  -81.8509   0.227006   0.115031
2  -83.1206   0.182116  0.0779065
3  -82.9737   0.198894  0.0590757
4  -82.9621   0.190459  0.0419953
5  -82.9397   0.186404  0.0375025
6  -82.7193   0.127346  0.0277753
7  -82.7128   0.146261  0.0235918
8  -82.4571   0.142665  0.0230555
9  -82.0935   0.131458  0.0208272
10 -81.6889   0.133016  0.0200892
11 -81.0395   0.142928  0.0187709
12 -80.3735   0.114991  0.0182263
13 -79.6306  0.0871631  0.0178589
14   -78.72  0.0849886  0.0167321
15 -77.6382  0.0889212  0.0160166
space
0.2071908405364502
space
83.70735287666321
space


In [0]:
from google.colab import drive
from math import sqrt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

drive.mount('/content/drive')
my_path = '/content/drive/My Drive/ecs171_yancey/Lecture_Notes/Chapter_3/YearPredictionMSD.txt'

# load CSV using pandas library
ms = pd.read_csv(my_path, header=None)
ms.head()
#print(ms.info())
sqrt(len(ms))

Y = ms.iloc[:,0]
X = ms.iloc[:,1:]
#newx = X.iloc[0:1000,:]
pcaOptimize(X,Y,200, 12, 5, 1.1)
# table, score, basetime = pcaCompare(X,Y,200,120,15)
# table[1] == table["Accuracy"]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
--- kNN without pca 94.75120449066162 seconds ---
--- kNN score without pca 0.28190375461133266 ---
0.25371337915019937
      Time  Accuracy   Variance
1 -91.8024    0.4367   0.115031
2  -93.964   0.31622  0.0779065
3 -93.9599  0.342466  0.0590757
4  -93.815  0.318696  0.0419954
5 -93.5523  0.338456  0.0375027
-93.96401572227478
Best PC for optimization is : 2 
